In [11]:
import cirq

from cirq import DensityMatrixSimulator, depolarize # selecting simulator and noise
from mitiq.interface import convert_to_mitiq
from mitiq.pec.representations.depolarizing import represent_operations_in_circuit_with_local_depolarizing_noise

from mitiq import pec, zne, ddd
from mitiq.zne.scaling import fold_gates_at_random


In [12]:
def execute(circuit, noise_level=0.2):
    """Returns Tr[ρ |0⟩⟨0|] (expectation value)where ρ is the state prepared by the circuit
    executed with depolarizing noise.
    """
    # Replace with code based on your frontend and backend.
    mitiq_circuit, _ = convert_to_mitiq(circuit)
    
    # We add a simple noise model to simulate a noisy backend, depolarizing noise
    noisy_circuit = mitiq_circuit.with_noise(depolarize(p=noise_level))
    
    rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
    #DensityMatrixSimulator(*,dtype,noise):simulator for density matrices and noisy quantum circuits
    return rho[0, 0].real #real value of matrix element (0,0)

In [13]:
a0 = cirq.q("a")
# 5,1,3
# 1 -> 5
# X X
ops4 = [cirq.X(a0), cirq.X(a0)]
noise_level = 0.2
factory = zne.RichardsonFactory([1.0, 2.0])
circuitA = cirq.Circuit(ops4)
noisy_valueA= execute(circuitA, noise_level)
ideal_valueA= execute(circuitA, noise_level=0.0)
rule = ddd.rules.xx
repsA = represent_operations_in_circuit_with_local_depolarizing_noise(circuitA, noise_level)

In [14]:
pec_executor = pec.mitigate_executor(execute, representations=repsA)
pec_valueA = pec_executor(circuitA)
print(f"Error without PEC:Circuit3: {abs(ideal_valueA - noisy_valueA) :.5f}")
print(f"Error with PEC:Circuit3:    {abs(ideal_valueA - pec_valueA) :.5f}")


Error without PEC:Circuit3: 0.23111
Error with PEC:Circuit3:    0.02888


In [15]:
zne_executor = zne.mitigate_executor(execute, factory=factory)
zne_valueA = zne_executor(circuitA)

print(f"Error without ZNE:Circuit3: {abs(ideal_valueA - noisy_valueA) :.5f}")
print(f"Error with ZNE:Circuit3:    {abs(ideal_valueA - zne_valueA) :.5f}")

Error without ZNE:Circuit3: 0.23111
Error with ZNE:Circuit3:    0.10682


/opt/homebrew/anaconda3/envs/qrize/lib/python3.11/site-packages/mitiq/zne/inference.py:96: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(


In [16]:
ddd_executor = ddd.mitigate_executor(executor=execute,rule = rule)
ddd_value = ddd_executor(circuitA)

print(f"Error without DDD:Circuit3: {abs(ideal_valueA - noisy_valueA) :.5f}")
print(f"Error with DDD:Circuit3:    {abs(ideal_valueA - ddd_value) :.5f}")

Error without DDD:Circuit3: 0.23111
Error with DDD:Circuit3:    0.23111


In [17]:
pec_ddd_combined_executor = ddd.mitigate_executor(executor=pec_executor,rule = rule)

In [18]:
zne_executor = zne.mitigate_executor(pec_executor, factory=factory)
combined_value = zne_executor(circuitA)

print(f"Error without COMBINED:Circuit3: {abs(ideal_valueA - noisy_valueA) :.5f}")
print(f"Error with COMBINED:Circuit3:    {abs(ideal_valueA - combined_value) :.5f}")

/opt/homebrew/anaconda3/envs/qrize/lib/python3.11/site-packages/mitiq/zne/inference.py:96: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(


In [ ]:
zne_executor = zne.mitigate_executor(pec_ddd_combined_executor, factory=factory)
combined_value = zne_executor(circuitA)

print(f"Error without COMBINED:Circuit3: {abs(ideal_valueA - noisy_valueA) :.5f}")
print(f"Error with COMBINED:Circuit3:    {abs(ideal_valueA - combined_value) :.5f}")

/opt/homebrew/anaconda3/envs/qrize/lib/python3.11/site-packages/mitiq/zne/inference.py:96: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(


Error without COMBINED:Circuit3: 0.23111
Error with COMBINED:Circuit3:    0.03459


In [ ]:
def launch(executor, types, circuit, **executor_args):
   zne_executor = zne.mitigate_executor(executor=, **executor_args)
   combined_value = zne_executor(circuit)

   print(f"Error without {types}:Circuit3: {abs(ideal_valueA - noisy_valueA) :.5f}")
   print(f"Error with {types}:Circuit3:    {abs(ideal_valueA - combined_value) :.5f}")